# Shadow Rollout with Seldon and Ambassador

This notebook shows how you can deploy "shadow" deployments to direct traffic not only to the main Seldon Deployment but also to a shadow deployment whose reponse will be dicarded. This allows you to test new models in a production setting and with production traffic and anlalyse how they perform before putting them live.

These are useful when you want to test a new model or higher latency inference piepline (e.g., with explanation components) with production traffic but without affecting the live deployment.


## Prerequistes

You will need

 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core) running this notebook
 - A running Kubernetes cluster with kubectl authenticated
 - [Helm client](https://helm.sh/)
 - Seldon Core Python Module : `pip install seldon-core`

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [2]:
!kubectl create namespace seldon

namespace/seldon created


In [1]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [4]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [5]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [6]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [2]:
!helm install ../../../helm-charts/seldon-core-operator --name seldon-core --set usageMetrics.enabled=true   --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Tue Apr 16 10:55:02 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                                        TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.111.111.87  <none>       443/TCP  0s

==> v1/StatefulSet
NAME                                DESIRED  CURRENT  AGE
seldon-operator-controller-manager  1        1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  1s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-

In [3]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


Install the Prometheus and Grafana example analytics

In [4]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false  --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Tue Apr 16 10:55:09 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     1s

==> v1/ServiceAccount
NAME        SECRETS  AGE
prometheus  1        1s

==> v1beta1/ClusterRole
NAME        AGE
prometheus  1s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  1s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           1s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-8rwbr      0/1    ContainerCreating  0         0s
alertmanager-deployment-7cd568f668-mn9xz  0/1    ContainerCreating  0         0s
grafana-prom-deployment-899b4dd7b-glkxl   0/1    ContainerCreating  0         0s
prometheus-node-exporter-cn2bt            0/1    Pending            0         0s
prometheus-deployment-7554c97586-6929p    0/1    

## Setup Ingress
Please note: There are reported gRPC issues with ambassador (see https://github.com/SeldonIO/seldon-core/issues/473).

In [6]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Tue Apr 16 10:55:37 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        1s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  1s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  1s

==> v1/Service
NAME               TYPE          CLUSTER-IP    EXTERNAL-IP  PORT(S)                     AGE
ambassador-admins  ClusterIP     10.111.3.211  <none>       8877/TCP                    1s
ambassador         LoadBalancer  10.111.26.70  <pending>    80:30093/TCP,443:30568/TCP  1s

==> v1/Deployment
NAME        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
ambassador  3        3        3           0          1s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-5b89d44544-98n89  0/1    ContainerCreating  0         1s
ambassador-5b89d44544-k4trg  0/1    ContainerCreating  0         1s
ambassador-5b89d44544-wpqkv  0/1    ContainerCreating  0

In [7]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


## Set up Port Forwards

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

**Ensure you port forward to Grafana**

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`.

In [17]:
!pygmentize model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example"
    },
    "spec": {
        "name": "production-model",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                                  ],
                "graph":
                {
                    "children": [],
                    "name": "classifier",
                    "type": "MODEL",
                    "endpoint": {
              

In [8]:
!kubectl create -f model.json

seldondeployment.machinelearning.seldon.io/example created


In [9]:
!kubectl rollout status deploy/production-model-single-7cd068f

Waiting for deployment "production-model-single-7cd068f" rollout to finish: 0 of 1 updated replicas are available...
deployment "production-model-single-7cd068f" successfully rolled out


### Get predictions

In [10]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="example",namespace="seldon")

#### REST Request

In [11]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.25103583044502875
  }
}

Response:
meta {
  puid: "j7mhk9clk2fbu8tdq3ka8m2274"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06503212230125832
  }
}



#### gRPC Request

In [12]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.6342191670710421
  }
}

Response:
meta {
  puid: "cb93sjuopd985fa8jde1bt023m"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09258712191804268
  }
}



## Launch Shadow

We will now create a new Seldon Deployment for our Shadow deployment with a new model `seldonio/mock_classifier_rest:1.1`. To make it a shadow of the original `example` deployment we add two annotations

```
"annotations": {
	    "seldon.io/ambassador-service-name":"example",
        "seldon.io/ambassador-shadow":"true"
	},	
```

The first says to use `example` as our service endpoint rather than the default which would be our deployment name - in this case `example-shadow`. This will ensure that this Ambassador setting will apply to the same prefix as the previous one. The second states we want to use Ambassador's shadow functionality.

In [13]:
!pygmentize shadow.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example-shadow"
    },
    "spec": {
        "name": "shadow-model",
	"annotations": {
	    "seldon.io/ambassador-service-name":"example",
	    "seldon.io/ambassador-shadow":"true"	    
	},	
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_rest:1.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                                  ],
                "graph":
                {
                    "children": [

In [14]:
!kubectl create -f shadow.json

seldondeployment.machinelearning.seldon.io/example-shadow created


In [15]:
!kubectl rollout status deploy/shadow-model-single-4c8805f

Waiting for deployment "shadow-model-single-4c8805f" rollout to finish: 0 of 1 updated replicas are available...
deployment "shadow-model-single-4c8805f" successfully rolled out


Let's send a bunch of requests to the endpoint.

In [16]:
for i in range(1000):
    r = sc.predict(gateway="ambassador",transport="rest")

Now view the analytics dashboard at http://localhost:3000 
You should see a dashboard view like below showing the two models production and shadow both receiving requests.

![shadow](shadow.png)
